# 환율 데이터 

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs


In [5]:
# 환율 데이터 수집
# 네이버 금융에서 환율 데이터를 수집해보자.
res = req.get("https://finance.naver.com/marketindex/")
res
# HTML 파싱  parser를 lxml이용해서 soup 객체로 변환
soup = bs(res.text, "lxml")
soup

<html><head><script language="javascript" src="/template/head_js.naver?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20250710182837/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20250710182837/js/jindo.1.5.3.element-text-patch.js" type="text/javascript"></script>
</head><body><div id="container" style="padding-bottom:0px;">
<div class="market_include">
<div class="market_data">
<div class="market1">
<div class="title">
<h2 class="h_market1"><span>환전 고시 환율</span></h2>
</div>
<!-- data -->
<div class="data">
<ul class="data_lst" id="exchangeList">
<li class="on">
<a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_up">
<span class="value">1,380.90</sp

In [13]:
# 통화명 수집
# copy selector로 통화명을 수집해보자.
# 주의할점 : 선택자는 완벽하지만, data가 안나온다 > 현재사이트에 데이터 접근이 불가능하다. 
# 통신은 잘 되었지만 content가 비어있다. iframe태그를 통해서 불러오겠다.  sorce src를 찾아라!!

test1 = soup.select("body > div > table > tbody > tr:nth-child(1) > td.tit > a")
test1
# iframe tag를 통해서 보여주는것일뿐이다. src = /marketindex/exchangeList.naver 상대경로를 나타내준다
# 절대경로로 바꿔준다. 


site_header = "https://finance.naver.com"
site_footer = "/marketindex/exchangeList.naver"

site = site_header + site_footer
print(site)


# request를 정확하게 해야한다. 
res = req.get(site)
soup = bs(res.text, "lxml")

# title = soup.select("body > div > table > tbody > tr:nth-child(1) > td.tit > a")
# 단수계의 선택자를 지우고싶을때 >> 뒤에서부터 단수를 지워나간다.  >> a는 복수 >> class는 복수개 >> nth-child()단수 (지워야한다. )  

title = soup.select("body > div > table > tbody > tr > td.tit > a")
title



https://finance.naver.com/marketindex/exchangeList.naver


[<a href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_USDKRW', '1', event);" target="_parent">
 				
 					
 					
 					
 					미국 USD
 				
 				</a>,
 <a href="/marketindex/exchangeDetail.naver?marketindexCd=FX_EURKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_EURKRW', '2', event);" target="_parent">
 				
 					
 					
 					
 					유럽연합 EUR
 				
 				</a>,
 <a href="/marketindex/exchangeDetail.naver?marketindexCd=FX_JPYKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_JPYKRW', '3', event);" target="_parent">
 				
 					일본 JPY (100엔)
 					
 					
 					
 				
 				</a>,
 <a href="/marketindex/exchangeDetail.naver?marketindexCd=FX_CNYKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_CNYKRW', '4', event);" target="_parent">
 				
 					
 					
 					
 					중국 CNY
 				
 				</a>,
 <a href="/marketindex/exchangeDetail.naver?marketindexCd=FX_HKDKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_HKDKRW', '5', 


### iframe정리 
- 응답이 200번이 넘어 왔느데, 수집이 안된다면 2가지 문제
1. 선택자를 잘못 작성한 경우
2. ifrmae 태그를 통해서 빌려오는경우

- iframe : 특정 페이지의 데이터를 여러 페이지에 빌려다가 사용하는 태그
- 장점 : 유지보수성, 재사용성이 좋다.
- 핵심속성 : sorce src=> 원본 데이터의 주소가 작성   ★★★blank를 통해서 숨겨버릴 수 있다. !!★★★ >> 셀레늄을 통해서 접근가능 
- 주소를 사용할 때 상대경로라면 url 절대경로로 변경할 것
- 변경된 url을 재 요청하기.

### 선택자 복사하는 방법
- 개발자도구 => 마우스 우클릭 => copy => copy selector
- 주의점 : 반드시 내가 선택한 태그 1개만 가지고오는 단수선택자가 리턴
- 활용법 : 복수개의 선택자가 필요 하다면, 단수를 의미하는걸 지우자(nth-child, id 등)
- 팁 : 선택자를 지울 때는 뒤에서부터 지워 나간다.
- 단! ★★★테이블★★★ 태그인 경우에는 반드시 주의해서 사용할 것 
- 핵심 : 테이블 태그에서 tr은 row를 의미하고, td는 column을 의미한다. 
- 팁 : 테이블 태그에서 nth-child를 삭제할 때는 반드시 tr만 지운다. 

In [17]:
# 매매기준율 수집
# soup.select("body > div > table > tbody > tr:nth-child(1) > td.sale")
trade_rate = soup.select("body > div > table > tbody > tr > td.sale")
trade_rate


[<td class="sale">1,382.30</td>,
 <td class="sale">1,612.31</td>,
 <td class="sale">938.36</td>,
 <td class="sale">192.79</td>,
 <td class="sale">176.09</td>,
 <td class="sale">47.19</td>,
 <td class="sale">1,860.44</td>,
 <td class="sale">3,590.58</td>,
 <td class="sale">1,009.46</td>,
 <td class="sale">1,732.21</td>,
 <td class="sale">143.95</td>,
 <td class="sale">906.65</td>,
 <td class="sale">826.34</td>,
 <td class="sale">65.38</td>,
 <td class="sale">1.45</td>,
 <td class="sale">34.38</td>,
 <td class="sale">0.39</td>,
 <td class="sale">411.45</td>,
 <td class="sale">216.08</td>,
 <td class="sale">135.97</td>,
 <td class="sale">368.57</td>,
 <td class="sale">4,523.97</td>,
 <td class="sale">3,666.77</td>,
 <td class="sale">376.34</td>,
 <td class="sale">1,949.65</td>,
 <td class="sale">27.94</td>,
 <td class="sale">42.64</td>,
 <td class="sale">1,078.62</td>,
 <td class="sale">324.75</td>,
 <td class="sale">8.50</td>,
 <td class="sale">379.27</td>,
 <td class="sale">2.64</td>,
 

In [19]:
# 사실때 가격 가져오기 . ★★★table★★★ 에서는 신경써야한다. 
# 뒤에서부터 지워야한다.  table data(column) 은그대로두고, table row(row)를 복수형으로 만들어야한다. 
# soup.select("body > div > table > tbody > tr:nth-child(1) > td:nth-child(3)")
sell_price = soup.select("body > div > table > tbody > tr > td:nth-child(3)")
sell_price


# body > div > table > tbody > tr:nth-child(1) > td:nth-child(4)

[<td>
 					
 						
 						1,406.49
 					
 				</td>,
 <td>
 					
 						
 						1,644.39
 					
 				</td>,
 <td>
 					
 						
 						954.78
 					
 				</td>,
 <td>
 					
 						
 						202.42
 					
 				</td>,
 <td>
 					
 						
 						179.55
 					
 				</td>,
 <td>
 					
 						
 						53.37
 					
 				</td>,
 <td>
 					
 						
 						1,897.09
 					
 				</td>,
 <td>
 					
 						
 						3,841.92
 					
 				</td>,
 <td>
 					
 						
 						1,029.34
 					
 				</td>,
 <td>
 					
 						
 						1,766.33
 					
 				</td>,
 <td>
 					
 						
 						148.70
 					
 				</td>,
 <td>
 					
 						
 						924.51
 					
 				</td>,
 <td>
 					
 						
 						842.61
 					
 				</td>,
 <td>
 					
 						
 						70.93
 					
 				</td>,
 <td>
 					
 						
 						1.59
 					
 				</td>,
 <td>
 					
 						N/A
 						
 					
 				</td>,
 <td>
 					
 						N/A
 						
 					
 				</td>,
 <td>
 					
 						
 						452.59
 					
 				</td>,
 <td>
 					
 						
 				

# 크롤링 오후 정리
1) 선택자 분석 => 클래스에 공백이 있는 경우
- 클래스가 복수개
- 해결책 공백을 지우고 "." 이어준다.
2) 개수가 중요한 데이터는 반드시 검증하자.
- 가수 100, 노래 100
- 데이터의 중복여부
3) 데이터 프레임 df 
- 표는 컬럼과 데이터 구조 => dictionary
- 파일로 저장(csv, excel로 저장할 때는 반드시 인코딩 하기 utf-8-sig ) browser가 알아서 인코딩 해준다.
4) iframe 태그
- 원본은 다른 서버에 저장해 두고, 필요할 때마다 불러다 사용한다. 
- 언제 ? 요청이 200으로 정상적인데, 데이터가 안나오면 iframe을 의심하자.
- 사용법 : iframe의 src sorce 속성에 원본 사이트의 주소가 작성
* 상대경로라면 url 절대경로로 수정하기
- 조합한 원본의 주소로 데이터 요청하기
5) 선택자 복사하는 방법
- 개발자도구 => 마우스 우클릭 => copy => copy selector
- 주의점 : 오직 내가 선택한 태그 한개만 반환하는 단수 선택자
- 복수가 필요하다면 반드시 단수의 의미를 삭제하자
- 반드시 뒤에서부터 단수의 의미를 지우자 (id, nth-child)
- 단 ! table 태그에서는 td(column)을 지우지말고, tr(row)를 지우자.
- 단수개를 의미하는걸 뒤에서부터 추가적으로 삭제
6) 내일 배울 내용
- selenium = > 크롤링의 핵심기술
- 모든 브라우저의 기능을 제어할 수 있는 라이브러리
* 사람처럼 코딩을 할 수 있다. 